# 02 - Feature Calculations with NumPy 


**Load the point clouds.**

In [1]:


filepath = './Vaihingen3DTraining.las'

print(filepath)

./Vaihingen3DTraining.las


**Extract and stack x,y,z from LAS file.**

In [2]:
import laspy
import numpy as np

file = laspy.read(filepath)

xyz = np.stack((file.x, file.y, file.z), axis=1)

print(xyz.shape)

(753876, 3)


**Compute eigenvalues and eigenvectors.**



In [3]:
print(xyz.shape)

(753876, 3)


In [4]:
from sklearn.neighbors import KDTree

# building the kd tree from the points of the point cloud
kd_tree_3d = KDTree(xyz)

# query for each point the k nearest neighbor points
distances3d, indices3d = kd_tree_3d.query(xyz, k=20)

# build the covariance matrix of neighbor points
xyz_minus_mean = xyz[indices3d] - np.mean(xyz[indices3d], axis=1)[:, np.newaxis, :]
xyz_xyz = np.matmul(np.transpose(xyz_minus_mean, axes=(0, 2, 1)), xyz_minus_mean)
xyz_cov = xyz_xyz / 19.0

# construct arrays to store eigenvalues and eigenvectors in
w = np.zeros((np.shape(xyz)[0], 3))
v = np.zeros((np.shape(xyz)[0], 3, 3))

for i in range(xyz.shape[0]):
    w[i], v[i,:] = np.linalg.eig(xyz_cov[i])
    
# sort eigenvalues and eigenvectors
idx = np.flip(np.argsort(w), axis=1)
w = np.take_along_axis(w, idx, axis=1)
v = np.take_along_axis(v, idx[:, :, np.newaxis], axis=1)    
    
# normalize eigenvalues
eigenvalues = w / np.sum(w, axis=-1)[:, np.newaxis]    
    
# make sure eigenvalues are greater than 0.0
eigenvalues[eigenvalues <= np.finfo(np.float32).eps] = np.finfo(np.float32).eps

# normalize the eigenvectors
eigenvectors = v / np.linalg.norm(v, axis=-1)[:,:,np.newaxis]    

In [5]:
print(w.shape)
print(v.shape)
print(np.sum(w, axis=-1)[:, np.newaxis].shape)

(753876, 3)
(753876, 3, 3)
(753876, 1)


# Mathematical (vectorized) functions in NumPy

# Eigenvalue based 3D features

In [6]:
# eigenvalues lambda 1
l1 = eigenvalues[:,0][:,np.newaxis]

# eigenvalues lambda 2
l2 = eigenvalues[:,1][:,np.newaxis]

# eigenvalues lambda 3
l3 = eigenvalues[:,2][:,np.newaxis]
print(l1.shape)

(753876, 1)


**Linearity:** $L_\lambda = \frac{\lambda_1 - \lambda_2}{\lambda_1}$


In [7]:
linearity = (l1-l2)/l1
print(linearity.shape)

(753876, 1)


**Planarity:** $P_\lambda = \frac{\lambda_2 - \lambda_3}{\lambda_1}$

In [8]:
planarity = (l2-l3)/l1
print(planarity.shape)

(753876, 1)


**Scattering** (or sphericity)**:** $S_\lambda = \frac{\lambda_3}{\lambda_1}$

In [9]:
scattering = l3/l1
print(scattering.shape)

(753876, 1)


**Omnivariance:** $O_\lambda = \sqrt[3]{\lambda_1 \lambda_2 \lambda_3}$

or $O_\lambda = \sqrt[3]{\prod\limits_{i=1}^3 \lambda_i}$


In [10]:
omnivariance = np.cbrt(l1*l2*l3)
print(omnivariance.shape)

(753876, 1)


**Anisotropy:** $A_\lambda = \frac{\lambda_1 - \lambda_3}{\lambda_1}$

In [11]:
anisotropy = (l1 - l3)/l1
print(anisotropy.shape)

(753876, 1)


**Eigenentropy:** $E_\lambda = -\sum\limits_{i=1}^3{\lambda_i \ln(\lambda_i) }$



In [12]:
eigenentropy = -(l1*np.log(l1) + l2*np.log(l2) + l3*np.log(l3))
print(eigenentropy.shape)

(753876, 1)


**Sum of eigenvalues:** $\sum_\lambda = \lambda_1 + \lambda_2 + \lambda_3$

or $\sum_\lambda = \sum\limits_{i=1}^3{\lambda_i}$



In [13]:

sum_eigenvalues = np.sum(w,axis=-1)[:,np.newaxis]
print(sum_eigenvalues.shape)

(753876, 1)


**Change of curvature** (local surface variation)**:** $C_\lambda = \frac{\lambda_3}{\lambda_1 + \lambda_2 + \lambda_3}$

or $C_\lambda = \frac{\lambda_3}{\sum\limits_{i=1}^3 \lambda_i}$

In [14]:
change_of_curvature = (l3/sum_eigenvalues)
print(change_of_curvature.shape)

(753876, 1)


# Geometric 3D features



**Radius of kNN (k nearest neighbors):** $r_{\text{kNN,3D}}$


In [15]:
radius_knn3d = distances3d[:,18:19]
print(radius_knn3d.shape)

(753876, 1)


**Local point density:** $D_{\text{3D}} = \frac{k+1}{\frac{4}{3}\pi r_{\text{kNN,3D}}^3}$



In [16]:
k = 19.0
density_3d = (k+1)/ (4/3)*np.pi*np.power(radius_knn3d,3.0)
print(density_3d.shape)

(753876, 1)


**Verticality:** $V = 1 - \lvert n_Z \rvert$



In [17]:
verticality = (1-np.abs(eigenvectors[:,2,2]))[:,np.newaxis]
print(verticality.shape)

(753876, 1)


**Absolute height:** $H = Z$



In [18]:
absolute_height = xyz[:,2:3]
print(absolute_height.shape)

(753876, 1)


**Height difference:** $\Delta H_\text{kNN,3D}$


In [19]:
print(xyz[indices3d][:,:,2:3].shape)
min_z =np.min(xyz[indices3d][:,:,2:3],axis =1)
max_z = np.max(xyz[indices3d][:,:,2:3],axis =1)
print(min_z.shape)
delta_z_knn3d = max_z - min_z
print(delta_z_knn3d.shape)

(753876, 20, 1)
(753876, 1)
(753876, 1)


**Standard deviation of height values:** $\sigma H_\text{kNN,3D}$


In [20]:
std_z = np.std(delta_z_knn3d,axis=1,ddof=1)[:,np.newaxis]
print(std_z.shape)
print(std_z.shape)

(753876, 1)
(753876, 1)


C:\Python310\lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Python310\lib\site-packages\numpy\core\_methods.py:258: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


# Eigenvalue based 2D features


In [21]:
print(xyz[:,0:2].shape)

(753876, 2)


In [22]:
xy= xyz[:,0:2]
# kd_tree_3d = KDTree(xy)

# query for each point the k nearest neighbor points
# distances2d, indices2d = kd_tree_3d.query(xy, k=20)

# build the covariance matrix of neighbor points
xy_minus_mean = xy[indices3d] - np.mean(xy[indices3d], axis=1)[:, np.newaxis, :]
xy_xy = np.matmul(np.transpose(xy_minus_mean, axes=(0, 2, 1)), xy_minus_mean)
xy_cov = xy_xy / 19.0
print(xy_cov.shape)

# construct arrays to store eigenvalues and eigenvectors in
w_2d = np.zeros((np.shape(xy)[0], 2))
v_2d = np.zeros((np.shape(xy)[0], 2, 2))

for i in range(xy.shape[0]):
    w_2d[i], v_2d[i,:] = np.linalg.eig(xy_cov[i])
    
# # sort eigenvalues and eigenvectors
idx = np.flip(np.argsort(w_2d), axis=1)
w_2d = np.take_along_axis(w_2d, idx, axis=1)
v_2d = np.take_along_axis(v_2d, idx[:, :, np.newaxis], axis=1)    
    
# # normalize eigenvalues
eigenvalues2d = w_2d / np.sum(w, axis=-1)[:, np.newaxis]    
    
# # make sure eigenvalues are greater than 0.0
eigenvalues[eigenvalues <= np.finfo(np.float32).eps] = np.finfo(np.float32).eps

# # normalize the eigenvectors
eigenvectors2d = v_2d / np.linalg.norm(v_2d, axis=-1)[:,:,np.newaxis] 
# eigenvalues2d = ...   
# eigenvectors2d = ... 
print(eigenvalues2d.shape)

(753876, 2, 2)
(753876, 2)


**Sum of eigenvalues:** $\sum_{\lambda\text{,2D}} = \lambda_\text{1,2D} + \lambda_\text{2,2D}$

Remember to use the unnormalized eigenvalues for this feature and not the normalized ones.

In [23]:

l1_2 = w_2d[:,0:1]
l2_2 = w_2d[:,1:2]
sum_of_eigenvalues_2d = l1_2 + l2_2
print(sum_of_eigenvalues_2d.shape) 

(753876, 1)


**Ratio of eigenvalues:** $R_{\lambda\text{,2D}} = \frac {\lambda_\text{2,2D}} {\lambda_\text{1,2D}}$

In [24]:
ratio_of_eigenvalues_2d = l2_2/l1_2
print(ratio_of_eigenvalues_2d.shape)

(753876, 1)


# Geometric 2D features



**Radius of kNN (k nearest neighbors):** $r_{\text{kNN,2D}}$ 




In [25]:
print(xyz[indices3d][:,:,0:2].shape)
print(xy[:,np.newaxis,:].shape)
dist_2d = xy[:,np.newaxis,:] - xyz[indices3d][:,:,0:2]
radius_knn_2d = (np.max(np.linalg.norm(dist_2d,axis =-1),axis = -1)[:,np.newaxis])
print(np.linalg.norm(dist_2d,axis =-1).shape)
print(radius_knn_2d.shape)


(753876, 20, 2)
(753876, 1, 2)
(753876, 20)
(753876, 1)


**Local point density:** $D_{\text{2D}} = \frac{k+1}{\pi r_{\text{kNN,2D}}^2}$

In [26]:
density_2d = ((k+1)/np.pi*radius_knn_2d**2.0)
print(density_2d.shape)

(753876, 1)


# Stack and save

In [27]:
stack_of_features = np.stack((linearity, 
                              planarity, 
                              scattering, 
                              omnivariance, 
                              anisotropy, 
                              eigenentropy, 
                              sum_eigenvalues, 
                              change_of_curvature, 
                              radius_knn3d, 
                              density_3d, 
                              verticality, 
                              absolute_height, 
                              delta_z_knn3d, 
                              std_z,
                              sum_of_eigenvalues_2d,
                              ratio_of_eigenvalues_2d,
                              radius_knn_2d,
                              density_2d,
                              l1,
                              l2,
                              l3,
                              l1_2,
                              l2_2
                             ), axis=1)

print(stack_of_features.shape)

(753876, 23, 1)


In [28]:
np.save('Features', stack_of_features)